In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.feature_selection import chi2

import argparse
import pandas as pd
import sys
import numpy as np
import pdb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import codecs
import operator
import gensim, sklearn
from collections import defaultdict
from nltk.tokenize import TweetTokenizer


In [ ]:
df=pd.read_csv("/kaggle/input/celebrity-profiling-training/celebrity_profiling_training_after_preprocessing_5_tweets.csv",usecols=['text','gender']) 
#,nrows=10000


In [ ]:
df.shape

In [ ]:

df['gender'].value_counts()

In [ ]:
df_majority = df[df.gender==1]
df_minority = df[df.gender==0]

In [ ]:
from sklearn.utils import resample

In [ ]:
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=206104,     # to match minority class
                                 random_state=123) # reproducible results

In [ ]:
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.gender.value_counts()

In [ ]:
df=df_downsampled

In [ ]:
del df_downsampled
del df_majority
del df_minority

In [ ]:
df.head()

In [ ]:
df=df.sample(frac=1)


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
tweets=df.text.tolist()

In [ ]:

labels = df.gender.tolist()


In [ ]:
text=[]
Y=[]
label0=[]
for i in range(len(tweets)):
    text.append(str(tweets[i]).lower())
    label0.append(int(labels[i]))


In [ ]:
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer
maxlen = 400

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text)

X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test,Y_train,Y_test= train_test_split(X, label0, test_size=0.2, random_state=42)

In [ ]:
print(tokenizer.word_index['for'])

In [ ]:
!pip install sister

In [ ]:
import sister

embedder  = sister.MeanEmbedding(lang="en")


In [ ]:
len(embedder("the"))

In [ ]:

def create_embedding_matrix(word_index, embedding_dim):
    
    
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word in word_index:
        try:
            idx = word_index[word]
            embedding_matrix[idx] = embedder(word)
        except:
            print(word)
    return embedding_matrix

In [ ]:
embedding_dim = 300

embedding_matrix = create_embedding_matrix(tokenizer.word_index, embedding_dim)

In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [ ]:
print("hachem")


def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]


    return embedding_matrix

embedding_dim = 300

embedding_matrix = create_embedding_matrix('/kaggle/input/wikinews300d1mvec/wiki-news-300d-1M.vec',tokenizer.word_index, embedding_dim)

embedding_dim = 200

embedding_matrix = create_embedding_matrix('/kaggle/input/gloveicg/glove/Glove/glove.6B.200d.txt',tokenizer.word_index, embedding_dim)

nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [ ]:
from keras.models import Sequential
from keras import layers


In [ ]:
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping


In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#model.add(Flatten())
model.add(Bidirectional(LSTM(100)))
#model.add(SimpleRNN(100))
#model.add(Dense(50, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict(X_test)
y_pred_conv = (y_pred.reshape(1, -1)[0] > .5) * 1

print("F1: %.2f%%" % (f1_score(Y_test, y_pred_conv, average='macro')*100))

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing import sequence

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict(X_test)
y_pred_conv = (y_pred.reshape(1, -1)[0] > .5) * 1

print("F1: %.2f%%" % (f1_score(Y_test, y_pred_conv, average='macro')*100))

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict(X_test)
y_pred_conv = (y_pred.reshape(1, -1)[0] > .5) * 1

print("F1: %.2f%%" % (f1_score(Y_test, y_pred_conv, average='macro')*100))

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict(X_test)
y_pred_conv = (y_pred.reshape(1, -1)[0] > .5) * 1

print("F1: %.2f%%" % (f1_score(Y_test, y_pred_conv, average='macro')*100))